In [1]:
import pandas as pd

pd.set_option('display.max_columns', 100)

def webscrape_scoring_data(season, week):
    qb = pd.read_html(f"https://www.footballguys.com/playerhistoricalstats?pos=qb&yr={season}&startwk={week}&stopwk={week}&profile=0")[0]
    qb.insert(1, 'Position', 'QB')
    qb.insert(0, 'Season', season)
    qb.insert(1, 'Week', week)

    rb = pd.read_html(f"https://www.footballguys.com/playerhistoricalstats?pos=rb&yr={season}&startwk={week}&stopwk={week}&profile=0")[0]
    rb.insert(1, 'Position', 'RB')
    rb.insert(0, 'Season', season)
    rb.insert(1, 'Week', week)
    rb['FantPt'] += rb['Rec']*0.5  # 1/2 ppr

    wr = pd.read_html(f"https://www.footballguys.com/playerhistoricalstats?pos=wr&yr={season}&startwk={week}&stopwk={week}&profile=0")[0]
    wr.insert(1, 'Position', 'WR')
    wr.insert(0, 'Season', season)
    wr.insert(1, 'Week', week)
    wr['FantPt'] += wr['Rec']*0.5  # 1/2 ppr

    te = pd.read_html(f"https://www.footballguys.com/playerhistoricalstats?pos=te&yr={season}&startwk={week}&stopwk={week}&profile=0")[0]
    te.insert(1, 'Position', 'TE')
    te.insert(0, 'Season', season)
    te.insert(1, 'Week', week)
    te['FantPt'] += te['Rec']*0.5  # 1/2 ppr

    dst = pd.read_html(f"https://www.footballguys.com/playerhistoricalstats?pos=td&yr={season}&startwk={week}&stopwk={week}&profile=fd")[0]
    dst.insert(1, 'Position', 'DST')
    dst.insert(0, 'Season', season)
    dst.insert(1, 'Week', week)

    k = pd.read_html(f"https://www.footballguys.com/playerhistoricalstats?pos=pk&yr={season}&startwk={week}&stopwk={week}&profile=fd")[0]
    k.insert(1, 'Position', 'K')
    k.insert(0, 'Season', season)
    k.insert(1, 'Week', week)
    k['FantPt'] += (k['XPM'] - k['XPA']) + (k['FGM'] - k['FGA'])  # deduct a point for missing fg or xp

    return pd.concat([qb, rb, wr, te, dst, k], ignore_index=True)

In [2]:
from itertools import product
from tqdm.autonotebook import tqdm

d = []
for season, week in tqdm(list(product([season for season in range(2020, 2024)], [week for week in range(1, 18)]))):
    if (week != 17) or (season >= 2021):
        d.append(webscrape_scoring_data(season, week))

C:\Users\mhugh\AppData\Local\Temp\ipykernel_13820\46214506.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/68 [00:00<?, ?it/s]

Combine dataframes

In [3]:
df = pd.concat(d, ignore_index=True)

df

,Season,Week,Rank,Position,Name,Age,Exp,G,Cmp,Att,Cm%,PYd,Y/Att,PTD,Int,Rsh,RshYd,RshTD,FP/G,FantPt,Y/Rsh,Rec,RecYd,RecTD,Y/Rec,SCK,FR,INT,DefTD,SpTmTD,PtAllw,YdAllw,FGM,FGA,FG%,XPM,XPA
0,2020,1,1,QB,Josh Allen BUF,24.0,3.0,1,33.0,46.0,71.7,312.0,6.78,2.0,0.0,14.0,57.0,1.0,32.2,32.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,1,2,QB,Russell Wilson SEA,32.0,9.0,1,31.0,35.0,88.6,322.0,9.20,4.0,0.0,3.0,29.0,0.0,31.8,31.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,1,3,QB,Aaron Rodgers GB,37.0,16.0,1,32.0,44.0,72.7,364.0,8.27,4.0,0.0,1.0,2.0,0.0,30.8,30.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,1,4,QB,Lamar Jackson BAL,23.0,3.0,1,20.0,25.0,80.0,275.0,11.00,3.0,0.0,7.0,45.0,0.0,27.5,27.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,1,5,QB,Kyler Murray ARI,23.0,2.0,1,26.0,40.0,65.0,230.0,5.75,1.0,1.0,13.0,91.0,1.0,26.3,26.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28828,2023,17,32,K,Lou Hedley NO,2023.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28829,2023,17,33,K,Dustin Hopkins CLE,33.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28830,2023,17,34,K,Eddy Piñeiro CAR,28.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28831,2023,17,35,K,Rigoberto Sanchez IND,29.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0


Extract team data from names

In [4]:
df.insert(df.columns.get_loc('Name') + 1, 'Team', df['Name'].str.split(' ').str[-1])
df['Name'] = df['Name'].str.split(' ').str[:-1].str.join(' ')

df

,Season,Week,Rank,Position,Name,Team,Age,Exp,G,Cmp,Att,Cm%,PYd,Y/Att,PTD,Int,Rsh,RshYd,RshTD,FP/G,FantPt,Y/Rsh,Rec,RecYd,RecTD,Y/Rec,SCK,FR,INT,DefTD,SpTmTD,PtAllw,YdAllw,FGM,FGA,FG%,XPM,XPA
0,2020,1,1,QB,Josh Allen,BUF,24.0,3.0,1,33.0,46.0,71.7,312.0,6.78,2.0,0.0,14.0,57.0,1.0,32.2,32.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,1,2,QB,Russell Wilson,SEA,32.0,9.0,1,31.0,35.0,88.6,322.0,9.20,4.0,0.0,3.0,29.0,0.0,31.8,31.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,1,3,QB,Aaron Rodgers,GB,37.0,16.0,1,32.0,44.0,72.7,364.0,8.27,4.0,0.0,1.0,2.0,0.0,30.8,30.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,1,4,QB,Lamar Jackson,BAL,23.0,3.0,1,20.0,25.0,80.0,275.0,11.00,3.0,0.0,7.0,45.0,0.0,27.5,27.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,1,5,QB,Kyler Murray,ARI,23.0,2.0,1,26.0,40.0,65.0,230.0,5.75,1.0,1.0,13.0,91.0,1.0,26.3,26.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28828,2023,17,32,K,Lou Hedley,NO,2023.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28829,2023,17,33,K,Dustin Hopkins,CLE,33.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28830,2023,17,34,K,Eddy Piñeiro,CAR,28.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28831,2023,17,35,K,Rigoberto Sanchez,IND,29.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0


Clean up data a bit before saving

In [5]:
df.drop(columns=['FP/G'], inplace=True)
df.rename(columns={'FantPt': 'Fantasy Points'}, inplace=True)

df.sort_values(by=['Season', 'Week', 'Position', 'Fantasy Points'], ascending=[True, True, False, False], inplace=True, ignore_index=True)

df

,Season,Week,Rank,Position,Name,Team,Age,Exp,G,Cmp,Att,Cm%,PYd,Y/Att,PTD,Int,Rsh,RshYd,RshTD,Fantasy Points,Y/Rsh,Rec,RecYd,RecTD,Y/Rec,SCK,FR,INT,DefTD,SpTmTD,PtAllw,YdAllw,FGM,FGA,FG%,XPM,XPA
0,2020,1,1,WR,Davante Adams,GB,28.0,7.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,34.6,NaN,14.0,156.0,2.0,11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,1,2,WR,Calvin Ridley,ATL,26.0,3.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-1.0,0.0,29.4,NaN,9.0,130.0,2.0,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,1,3,WR,Adam Thielen,MIN,30.0,8.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,26.0,NaN,6.0,110.0,2.0,18.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,1,4,WR,Darius Slayton,NYG,23.0,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,25.2,NaN,6.0,102.0,2.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,1,11,WR,DeAndre Hopkins,ARI,28.0,8.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,22.1,NaN,14.0,151.0,0.0,10.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28828,2023,17,28,DST,Tampa Bay Buccaneers,TB,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,23.0,310.0,NaN,NaN,NaN,NaN,NaN
28829,2023,17,29,DST,Washington Commanders,WAS,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,27.0,408.0,NaN,NaN,NaN,NaN,NaN
28830,2023,17,30,DST,Seattle Seahawks,SEA,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,30.0,468.0,NaN,NaN,NaN,NaN,NaN
28831,2023,17,31,DST,Atlanta Falcons,ATL,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,37.0,432.0,NaN,NaN,NaN,NaN,NaN


In [7]:
df.loc[df['Position'] == 'K', :]

,Season,Week,Rank,Position,Name,Team,Age,Exp,G,Cmp,Att,Cm%,PYd,Y/Att,PTD,Int,Rsh,RshYd,RshTD,Fantasy Points,Y/Rsh,Rec,RecYd,RecTD,Y/Rec,SCK,FR,INT,DefTD,SpTmTD,PtAllw,YdAllw,FGM,FGA,FG%,XPM,XPA
388,2020,1,1,K,Daniel Carlson,LV,25.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,100.0,4.0,4.0
389,2020,1,2,K,Mason Crosby,GB,36.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,100.0,5.0,5.0
390,2020,1,3,K,Josh Lambo,JAX,30.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,100.0,3.0,3.0
391,2020,1,4,K,Matt Prater,DET,36.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,75.0,2.0,2.0
392,2020,1,5,K,Joey Slye,CAR,24.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,100.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28796,2023,17,32,K,Lou Hedley,NO,2023.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28797,2023,17,33,K,Dustin Hopkins,CLE,33.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28798,2023,17,34,K,Eddy Piñeiro,CAR,28.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
28799,2023,17,35,K,Rigoberto Sanchez,IND,29.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0


Save data

In [6]:
df.to_parquet('../../data/fantasy_points/footballguys_half_ppr.parquet')

print('Done')

Done
